In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


## 분야별 클러스터링

In [3]:
# !pip install konlpy
# !pip tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 1.3 MB/s 
     |████████████████████████████████| 453 kB 37.8 MB/s 
ERROR: unknown command "tqdm"


In [71]:
import pandas as pd
import numpy as np

import konlpy
from tqdm import tqdm
from konlpy.tag import Okt

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.cluster import DBSCAN

In [72]:

def preprocessing_clustering(df, part):

  if part == 'int':
    df = df[df['분야'] == part]
  else:
    df = df[df['분야'] != part]
    


  df = df.drop_duplicates(['타이틀'], keep = 'first')
  # df.shape

  df['text50'] = np.nan
  df['text15'] = np.nan

  for index, row in df.iterrows():
    if len(str(df['본문'][index])) >= 50:
      df['text50'][index] = str(df['본문'][index])[:50]
    else:
      df['text50'][index] = str(df['본문'][index])

    if len(str(df['본문'][index])) >= 15:
      df['text15'][index] = str(df['본문'][index])[:15]
    else:
      df['text15'][index] = str(df['본문'][index])

  df = df.drop_duplicates(subset = 'text15', keep = 'first')

  regions = ['뉴스핌', '연예', '서울', '경기', '강원', '충청', '경상', '전라', '인천', '경남', '경북', '대구', '울산', '부산', '광주', '전남', '제주', '전북', '대전', '세종', '충남', '충북', '충청북도', '충청남도', '경상남도', '경상북도', '전라북도', '전라남도']


  drop_index_list = [] # 지워버릴 index를 담는 리스트
  for i, row in df.iterrows():
      temp_region = row['text15']
      # temp_region = row['본문'].split(".")[0]
      for region in regions:
        if region in temp_region: 
            drop_index_list.append(i)
  df = df.drop(drop_index_list) # 해당 index를 지우기
  # doc.shape


  okt = Okt() # 형태소 분석기 객체 생성
  noun_list = []
  for content in tqdm(df['text50']): 
      nouns = okt.nouns(content) # 명사만 추출하기, 결과값은 명사 리스트
      noun_list.append(nouns)

  df['nouns'] = noun_list


  # 명사리스트 비어있는 경우 삭제

  drop_index_list = [] # 지워버릴 index를 담는 리스트
  for i, row in df.iterrows():
      temp_nouns = row['nouns']
      if len(temp_nouns) == 0: # 만약 명사리스트가 비어 있다면
          drop_index_list.append(i) # 지울 index 추가
          
  df = df.drop(drop_index_list) # 해당 index를 지우기

  # index를 지우면 순회시 index 값이 중간중간 비기 때문에 index를 다시 지정
  df.index = range(len(df))

  if part == 'int':
    min_df = 6
    min_samples = 5
  elif df.shape[0] < 10000:
    min_df = 5
    min_samples = 5
  else:
    min_df = 6
    min_samples = 5


  # 문서를 명사 집합으로 보고 문서 리스트로 치환 (tfidfVectorizer 인풋 형태를 맞추기 위해)
  text = [" ".join(noun) for noun in df['nouns']]

  #|tfidf_vectorizer = TfidfVectorizer(4567\778qqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqq, ngram_range=(1,4))
  tfidf_vectorizer.fit(text)
  vector = tfidf_vectorizer.transform(text).toarray()


  vector = np.array(vector) # Normalizer를 이용해 변환된 벡터
  model = DBSCAN(eps=0.7,min_samples=min_samples, metric = "cosine")
  # 거리 계산 식으로는 Cosine distance를 이용
  result = model.fit_predict(vector)

  
  df['result'] = result
  if part == 'int':
    df['result'] = df['result'] + 100


  print(f"\n {df['result'].unique()} \n")

  return df

In [73]:
def cluster_result(df):

  cluster_idx = {}
  cluster_nums = {}
  for cluster_num in set(df['result']):
      # -1,0은 노이즈 판별이 났거나 클러스터링이 안된 경우
      if(cluster_num == -1 or cluster_num == 0 or cluster_num == 99 or cluster_num == 100): 
          continue
      else:
          print("cluster num : {}".format(cluster_num))
        
          temp_df = df[df['result'] == cluster_num] # cluster num 별로 조회
          num = len(df.index[df['result'] == cluster_num].to_list())
          for title in temp_df['타이틀']:
              print(title)
          cluster_idx[cluster_num] = df.index[df['result'] == cluster_num].to_list()
          cluster_nums[cluster_num] = num

          print("docs_num : {}".format(num))

          print()


  print(cluster_idx)

  return cluster_nums

In [75]:
df = pd.read_excel("C:/phoenix/news_selection/20220829_all.xlsx") # , encoding = "CP949"
df.head()

,발행일자,분야,타이틀,링크,본문
0,2022-08-29,pol,"신평 ""이제 밝힌다 '尹安 단일화' 나와 조정관 교수…安이 두사람 신뢰""",https://news.nate.com/view/20220829n19947?mid=...,지난 3월 3일 오전 윤석열 국민의힘 대선 후보와 안철수 국민의당 대선 후보가 서울...
1,2022-08-29,pol,"""치유와 위로 시간이었다""…평산마을 음악회 즐긴 文부부",https://news.nate.com/view/20220829n05205?mid=...,문재인 전 대통령이 거주중인 경남 양산 하북면 평산마을에서 주민들과 함께 퇴임 10...
2,2022-08-29,pol,"권성동 ""서병수 전국위 소집 거부할 경우, 부의장 대신 사회""",https://news.nate.com/view/20220829n32998?mid=...,권성동 국민의힘 원내대표는 29일 서병수 전국위원회 의장이 새 비대위 구성 절차인 ...
3,2022-08-29,pol,"정무 1·2 비서관 동반 사의…대통령실 ""국민관점에서 끊임없이 보완 할 것""",https://news.nate.com/view/20220829n33572?mid=...,"대통령실 ""국민에게 봉사, 헌신 해야하는 대통령실에 맞게 역량 갖출 인사 보완 할 ..."
4,2022-08-29,pol,"[지지율 분석] 尹, 3주 연속 상승했지만…더 이탈한 중도층",https://news.nate.com/view/20220829n33602?mid=...,정당 지지율은 與野 모두 소폭 상승윤석열 대통령이 29일 오전 서울 용산 대통령실 ...


In [76]:
df.shape

(16712, 5)

In [77]:
part_name = ['all'] # , 'int'

news_df = pd.DataFrame()

for part in part_name:
  df_result = preprocessing_clustering(df, part)
  news_df = pd.concat([news_df,df_result])


C:\Users\043\AppData\Local\Temp\ipykernel_64524\4285729156.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text50'][index] = str(df['본문'][index])[:50]
C:\Users\043\AppData\Local\Temp\ipykernel_64524\4285729156.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text15'][index] = str(df['본문'][index])[:15]
100%|██████████| 8765/8765 [00:40<00:00, 214.09it/s]



 [ -1   0   4   1   2   3  24  17   5 178   6   8   7   9 151  10  11  16
 123  12  13  14 120 156  15  63  18  19  20  21  22  23 154 118  26 157
  27  25  28  29  68  30  31  93  32  33  34  35  36  37 161  70  38 162
  39  40  41  42  62  43  44  45  46 104  47  84  96  85  92  48  60  49
  50  51  52  79  53  54  55  56  57  58 159  59  78  61  65  64  74  95
  97 106 176  66  67  69  76 152  71  72  73  75 107  86 109  77 124  80
  81  91  82  83 126 155  87  88 108  89  90  98 102 125  94 103 111  99
 146 112 100 101 184 105 182 121 110 113 170 168 179 114 140 115 116 141
 163 147 117 119 135 142 165 137 167 131 122 139 144 132 136 127 128 129
 160 130 133 150 134 158 166 164 145 149 148 143 138 169 153 171 181 172
 177 173 180 174 175 183] 



In [78]:
news_df['result'].unique()

array([ -1,   0,   4,   1,   2,   3,  24,  17,   5, 178,   6,   8,   7,
         9, 151,  10,  11,  16, 123,  12,  13,  14, 120, 156,  15,  63,
        18,  19,  20,  21,  22,  23, 154, 118,  26, 157,  27,  25,  28,
        29,  68,  30,  31,  93,  32,  33,  34,  35,  36,  37, 161,  70,
        38, 162,  39,  40,  41,  42,  62,  43,  44,  45,  46, 104,  47,
        84,  96,  85,  92,  48,  60,  49,  50,  51,  52,  79,  53,  54,
        55,  56,  57,  58, 159,  59,  78,  61,  65,  64,  74,  95,  97,
       106, 176,  66,  67,  69,  76, 152,  71,  72,  73,  75, 107,  86,
       109,  77, 124,  80,  81,  91,  82,  83, 126, 155,  87,  88, 108,
        89,  90,  98, 102, 125,  94, 103, 111,  99, 146, 112, 100, 101,
       184, 105, 182, 121, 110, 113, 170, 168, 179, 114, 140, 115, 116,
       141, 163, 147, 117, 119, 135, 142, 165, 137, 167, 131, 122, 139,
       144, 132, 136, 127, 128, 129, 160, 130, 133, 150, 134, 158, 166,
       164, 145, 149, 148, 143, 138, 169, 153, 171, 181, 172, 17

In [79]:
news_df.head(3)

,발행일자,분야,타이틀,링크,본문,text50,text15,nouns,result
0,2022-08-29,pol,"신평 ""이제 밝힌다 '尹安 단일화' 나와 조정관 교수…安이 두사람 신뢰""",https://news.nate.com/view/20220829n19947?mid=...,지난 3월 3일 오전 윤석열 국민의힘 대선 후보와 안철수 국민의당 대선 후보가 서울...,지난 3월 3일 오전 윤석열 국민의힘 대선 후보와 안철수 국민의당 대선 후보가 서울...,지난 3월 3일 오전 윤석열,"[지난, 오전, 윤석열, 국민, 힘, 대선, 후보, 안철수, 국민의당, 대선, 후보...",-1
1,2022-08-29,pol,"""치유와 위로 시간이었다""…평산마을 음악회 즐긴 文부부",https://news.nate.com/view/20220829n05205?mid=...,문재인 전 대통령이 거주중인 경남 양산 하북면 평산마을에서 주민들과 함께 퇴임 10...,문재인 전 대통령이 거주중인 경남 양산 하북면 평산마을에서 주민들과 함께 퇴임 10...,문재인 전 대통령이 거주중인,"[문재인, 전, 대통령, 거주, 경남, 양산, 하북면, 평산, 마을, 주민, 퇴임]",0
2,2022-08-29,pol,"권성동 ""서병수 전국위 소집 거부할 경우, 부의장 대신 사회""",https://news.nate.com/view/20220829n32998?mid=...,권성동 국민의힘 원내대표는 29일 서병수 전국위원회 의장이 새 비대위 구성 절차인 ...,권성동 국민의힘 원내대표는 29일 서병수 전국위원회 의장이 새 비대위 구성 절차인 ...,권성동 국민의힘 원내대표는,"[권성동, 국민, 힘, 원내대표, 서병수, 전국, 위원회, 의장, 새, 비, 대위,...",0


In [80]:
cluster_nums = cluster_result(news_df)

cluster num : 1
"피바람 불고 있다"…대통령실, 정무 1·2 비서관 사실상 경질
고강도 인적쇄신 나선 윤 대통령, 정무비서관 사실상 경질
정무 1·2비서관 동시 사퇴…윤핵관 라인 솎아내기?
野박수현 "김건희 특검 추진? 마음 알지만 과유불급"
[자막뉴스] "尹, 한계치 다다른 듯"…용산 칼바람 '폭풍전야'
"'고생 많았다, 그만 나가달라' 한밤 전화에 2명 용산 나갔다"
尹 "유능·헌신적 집단돼야"…대통령실 '폭풍전야'
대통령실 정무라인부터 쇄신…시민사회수석실 개편 가능성
정무 비서관 2명 사의…대통령실 인적쇄신 속도
대통령실 비서관 3명 물러났다…날카로워지는 '물갈이' 칼날
대통령실 정무1·2 비서관 사의…여당 내홍 관련 경질 성격 해석
대통령실 비서관 2명 자진사퇴…인적쇄신 속도
대통령실 정무1·2비서관 동시 사퇴…"끊임없이 보완"
대통령실  "정무 1·2 비서관 자진사퇴…수석도 예외없다"
대통령실 정무라인 사의…사실상 경질 관측
尹대통령 "野 포함 국회와 일해야…여야, 민생 위해 하나 돼야"
대통령실 정무라인 손본다…홍지만·경윤호 정무비서관 사의
[단독]대통령실, 홍지만·경윤호 정무비서관 사실상 경질 가닥
尹 "대통령실, 가장 헌신적이고 유능한 집단 돼야"
대통령실, '인적쇄신'은 계속된다…'윤핵관' 힘빼기 본격화되나
[속보]윤대통령 "정부 긴축하지만, 미래투자 등 돈 쓸때는 확실히 쓰겠다"
[속보]尹대통령, 인적쇄신 추진에 "대통령실은 가장 유능한 집단돼야"
'인사개입' 감찰받던 비서관, 자진 사퇴 의사
docs_num : 23

cluster num : 2
소방공무원이 봉사활동 하는 김건희? 김 여사 팬클럽 또 구설
소방공무원에 동그라미 치고 '건희 여사'…건희사랑 또 논란
수해복구 공무원에 동그라미 치고 '건희 여사'…건희사랑, 이번엔 사진조작 '구설'
김건희 여사 팬카페, 이번엔 '봉사활동 사진 조작' 논란
수해복구 공무원을 김건희 여사로…건희사랑, 이번엔 사진조작 '구설'
표절과 학문의 유지[김우재의 플라이룸]
김건희 여사 봉사활동 

In [81]:
def top10(news_df, cluster_nums):
  result_clus = sorted(cluster_nums.items(), key = lambda x:x[1], reverse = True)[:10]
  print(result_clus)

  fin_df = pd.DataFrame()
  clus_group = []
  for clus in result_clus:
    clus_group.append(clus[0])
    print(f"{clus[0]}---------------------------------------")
    temp_df = news_df[news_df['result'] == clus[0]] # cluster num 별로 조회
    print(temp_df['본문'])

    fin_df = pd.concat([fin_df, temp_df])
  
  return fin_df, clus_group

In [117]:
fin_df, clus_group = top10(news_df, cluster_nums)
fin_df['result'].unique()

[(26, 89), (116, 39), (11, 33), (33, 33), (84, 31), (52, 28), (155, 24), (1, 23), (48, 22), (111, 22)]
26---------------------------------------
1203    위원장에 강형구 의원·부위원장에 신정란 의원 선출[순천=뉴스핌] 오정근 기자 = 전...
1259    수산물 밀수·밀입국 등 범죄 예방…외사활동 강화[목포=뉴스핌] 김대원 기자 = 목포...
1396    면 요리 프로젝트 그룹 RPJ, 첫 디지털 싱글 '멋진삼총사' 발매[서울=뉴스핌] ...
1416    9월 14일까지 온라인 투표[서울=뉴스핌] 조용준 기자 = 문화체육관광부와 한국콘텐...
1645    이병호 농어촌公 사장, 비전 선포“식량안보, 기후위기, 농어촌 공간재생과 같은 농어...
                              ...                        
7332    보물 제1494호인 매천 황현 초상화. 사진=광양시 제공[광양(전남)=데일리한국 정...
7365    여수 돌산도 해역에 적조 주의보가 새로 발령됐다. 사진=전남도 제공[전남=데일리한국...
7437    문화체육관광부 [파이낸셜뉴스] 문화체육관광부는 박물관 및 미술관 진흥법에 따라 공립...
7578    상황실 운영하며 예찰 강화전남도는 지난 26일 오후 6시 고흥 외나로도에서 여수 돌...
7735    추석을 앞두고 명절 선물을 저렴하게 구매할 수 있는 ‘2022 추석맞이 우수중소기업...
Name: 본문, Length: 89, dtype: object
116---------------------------------------
4958    군위군청 전경. 사진=군위군 제공[군위(경북)=데일리한국 김원균 기자]경북 군위군의...
4962    김종윤 교육장이 지난 26일 열린 연수회에서 위촉장을 전달하고 있다. 사진=상주교육...
4965    거꾸로 멘토링 행사에서 참

array([ 26, 116,  11,  33,  84,  52, 155,   1,  48, 111], dtype=int64)

#### 클러스터링 된 집합에서 대표 뉴스 선정
- [] 랜덤

In [83]:
fin_df.head(3)

,발행일자,분야,타이틀,링크,본문,text50,text15,nouns,result
1203,2022-08-29,pol,순천시의회 '의과대학 유치 지원' 특별위원회 구성,https://news.nate.com/view/20220829n10541?mid=...,위원장에 강형구 의원·부위원장에 신정란 의원 선출[순천=뉴스핌] 오정근 기자 = 전...,위원장에 강형구 의원·부위원장에 신정란 의원 선출[순천=뉴스핌] 오정근 기자 = 전...,위원장에 강형구 의원·부위원,"[위원장, 강형구, 의원, 부위, 원장, 정란, 의원, 선출, 순천, 뉴스, 오정근...",26
1259,2022-08-29,pol,"목포해경, 추석 앞두고 먹거리 불법유통 집중 단속",https://news.nate.com/view/20220829n09396?mid=...,수산물 밀수·밀입국 등 범죄 예방…외사활동 강화[목포=뉴스핌] 김대원 기자 = 목포...,수산물 밀수·밀입국 등 범죄 예방…외사활동 강화[목포=뉴스핌] 김대원 기자 = 목포...,수산물 밀수·밀입국 등 범죄,"[수산물, 밀수, 밀, 입국, 등, 범죄, 예방, 활동, 강화, 목포, 뉴스, 김대...",26
1396,2022-08-29,pol,라면·파스타·자장면 모티브의 캐릭터 그룹 RPJ 음식 주제 댄스곡 발매,https://news.nate.com/view/20220829n05209?mid=...,"면 요리 프로젝트 그룹 RPJ, 첫 디지털 싱글 '멋진삼총사' 발매[서울=뉴스핌] ...","면 요리 프로젝트 그룹 RPJ, 첫 디지털 싱글 '멋진삼총사' 발매[서울=뉴스핌] ...",면 요리 프로젝트 그룹 RP,"[면, 요리, 프로젝트, 그룹, 첫, 디지털, 싱글, 삼총사, 발매, 서울, 뉴스,...",26


In [114]:
for idx in fin_df['본문'].index:
    if len(fin_df['본문'][idx]) < 1024:
        fin_df = fin_df.drop(idx)

344
93


In [121]:
## 클러스터별 랜덤으로 1개 추출
import random

def final_dataset(fin_df, clus_group):

  # fin_df 인덱스 재설정
  fin_df.index = range(len(fin_df))
  # final_news = pd.DataFrame()
  idx = []
  for clus in clus_group:
    temp_idx = fin_df[fin_df['result'] == clus].index
    idx.append(temp_idx[random.randrange(0, len(temp_idx))])
    # print(temp_idx)

  # print(idx)
  final_news = fin_df.iloc[idx, :5]

  return final_news



In [119]:
final_news = final_dataset(fin_df, clus_group)
final_news

,발행일자,분야,타이틀,링크,본문,text50,text15,nouns,result
53,2022-08-29,soc,2022 고흥 남열 전국 서핑대회 성료,https://news.nate.com/view/20220829n22797?mid=...,전국 300여명 서퍼들의 묘기와 페스티벌 한마당 행사 열려[고흥=아시아경제 호남취재...,전국 300여명 서퍼들의 묘기와 페스티벌 한마당 행사 열려[고흥=아시아경제 호남취재...,전국 300여명 서퍼들의 묘,"[전국, 여명, 서퍼, 묘기, 페스티벌, 마당, 행사, 고흥, 아시아, 경제, 호남...",26
92,2022-08-29,soc,"상주시, 추석 앞두고 불법 주·정차 단속 한시적 유예",https://news.nate.com/view/20220829n37020?mid=...,상주시청 전경. 사진=데일리한국DB[상주(경북)=데일리한국 김철희 기자] 경북 상주...,상주시청 전경. 사진=데일리한국DB[상주(경북)=데일리한국 김철희 기자] 경북 상주...,상주시청 전경. 사진=데일리,"[상주시청, 전경, 사진, 데, 일리, 한국, 상주, 경북, 데, 일리, 한국, 김...",116
130,2022-08-29,eco,50년만의 달탐사…美 '아르테미스' 발사 연기,https://news.nate.com/view/20220829n36967?mid=...,미국이 반세기 만에 재개한 유인 달탐사 '아르테미스 프로그램'의 첫 시험비행이 오는...,미국이 반세기 만에 재개한 유인 달탐사 '아르테미스 프로그램'의 첫 시험비행이 오는...,미국이 반세기 만에 재개한,"[미국, 반세기, 재개, 유인, 달, 탐사, 아르테미스, 프로그램, 의, 첫, 시험...",11
164,2022-08-29,eco,"KB국민은행, '투자자 유형 분류 서비스' 도입",https://news.nate.com/view/20220829n29575?mid=...,"고객 투자스타일, 16개 유형으로 분류""자산관리 서비스 세분화""KB국민은행이 KB스...","고객 투자스타일, 16개 유형으로 분류""자산관리 서비스 세분화""KB국민은행이 KB스...","고객 투자스타일, 16개 유","[고객, 투자, 스타일, 개, 유형, 분류, 자산, 관리, 서비스, 세분, 국민은행...",33
204,2022-08-29,eco,"[혁신경영] ""일하는 시간도 내 스타일대로"" CJ대한통운, 스마트 선택근무제 도입",https://news.nate.com/view/20220829n01369?mid=...,[이투데이/남주현 기자]강신호 대표가 메타버스를 통해 직원들과 소통하고 있다.(사진...,[이투데이/남주현 기자]강신호 대표가 메타버스를 통해 직원들과 소통하고 있다.(사진...,[이투데이/남주현 기자]강신,"[이투데이, 남주현, 기자, 강신호, 대표, 메타, 버스, 통해, 직원, 소통, 사...",84
237,2022-08-29,soc,"30년 발자취 따라…동서대, 개교 30주년 기념 공모전 수상작 상영회",https://news.nate.com/view/20220829n29069?mid=...,[아시아경제 영남취재본부 황두열 기자] 동서대학교가 오는 9월 1일~3일 사흘간 개...,[아시아경제 영남취재본부 황두열 기자] 동서대학교가 오는 9월 1일~3일 사흘간 개...,[아시아경제 영남취재본부 황,"[아시아, 경제, 영남, 취재, 본부, 황, 열, 기자, 동서대학교, 사흘, 개교]",52
269,2022-08-29,soc,"""짧고 강력하게, 60초 내 영상에 종로 매력 담아주세요""",https://news.nate.com/view/20220829n12542?mid=...,60초 영상 공모전(종로구 제공).(서울=뉴스1) 전준우 기자 = 서울 종로구는 최...,60초 영상 공모전(종로구 제공).(서울=뉴스1) 전준우 기자 = 서울 종로구는 최...,60초 영상 공모전(종로구,"[영상, 공모전, 종로구, 제공, 서울, 뉴스, 전준우, 기자, 서울, 종로구, 최근]",155
292,2022-08-29,pol,"尹대통령 ""野 포함 국회와 일해야…여야, 민생 위해 하나 돼야""",https://news.nate.com/view/20220829n09778?mid=...,"대통령실 내부 감찰 관련 ""대통령실, 업무 역량 늘 최고조 유지돼야""與 비대위 재구...","대통령실 내부 감찰 관련 ""대통령실, 업무 역량 늘 최고조 유지돼야""與 비대위 재구...","대통령실 내부 감찰 관련 ""","[대통령실, 내부, 감찰, 관련, 대통령실, 업무, 역량, 늘, 최고, 유지, 비,...",1
315,2022-08-29,eco,"소리바다, 정리매매 첫날 88% 급락",https://news.nate.com/view/20220829n06295?mid=...,- [특징주][이데일리 양지윤 기자] 상장폐지를 앞둔 소리바다가 정리매매 개시 첫날...,- [특징주][이데일리 양지윤 기자] 상장폐지를 앞둔 소리바다가 정리매매 개시 첫날...,- [특징주][이데일리 양지,"[특징, 주, 이데일리, 지윤, 기자, 상장폐지, 소리바다, 정리, 매매, 개시, 첫날]",48
343,2022-08-29,int,"""너 대체 뭐 될래"" 타박 받던 '젤리군', 年330억 사장된 비결",https://news.nate.com/view/20220829n28431?mid=...,"얼핏 보면 업력 수십 년은 거뜬히 됐을 듯한 노포(老鋪)인데, 창업년도는 2019년...","얼핏 보면 업력 수십 년은 거뜬히 됐을 듯한 노포(老鋪)인데, 창업년도는 2019년...",얼핏 보면 업력 수십 년은,"[얼핏, 업력, 수십, 년, 듯, 노포, 창업, 년도, 일본]",111


In [172]:
final_news.to_excel("/content/drive/MyDrive/Colab Notebooks/clustered_0826.xlsx", encoding = "utf-8")